In [1]:
import Metrics
import Models
import os, tqdm
import numpy as np
from Encoder import Encoder
from Binarizer import Binarizer
from datasetGenerator import DCASE2018


# evaluate
from evaluation_measures import event_based_evaluation
from dcase_util.containers import MetaDataContainer

from keras.models import model_from_json, Model, load_model
from keras.layers import GRU, Bidirectional, Layer, TimeDistributed, Dense, GRUCell
from keras import backend as K
from keras import regularizers


Using TensorFlow backend.


In [2]:
class CustomGRUCell(GRUCell):

    def __init__(self, units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True,
                 kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros',
                 kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, kernel_constraint=None,
                 recurrent_constraint=None, bias_constraint=None, dropout=0., recurrent_dropout=0., implementation=1,
                 reset_after=False, temporal_weight: float = 0.5, **kwargs):

        self.temporal_weight = temporal_weight

        super().__init__(units, activation, recurrent_activation, use_bias, kernel_initializer, recurrent_initializer,
                         bias_initializer, kernel_regularizer, recurrent_regularizer, bias_regularizer,
                         kernel_constraint, recurrent_constraint, bias_constraint, dropout, recurrent_dropout,
                         implementation, reset_after, **kwargs)
        
        print("Temporal weight : ", self.temporal_weight)


    def call(self, inputs, states, training=None):
        h_tm1 = states[0]  # previous memory

        # if 0 < self.dropout < 1 and self._dropout_mask is None:
        #     self._dropout_mask = _generate_dropout_mask(
        #         K.ones_like(inputs),
        #         self.dropout,
        #         training=training,
        #         count=3)
        # if (0 < self.recurrent_dropout < 1 and
        #         self._recurrent_dropout_mask is None):
        #     self._recurrent_dropout_mask = _generate_dropout_mask(
        #         K.ones_like(h_tm1),
        #         self.recurrent_dropout,
        #         training=training,
        #         count=3)

        # dropout matrices for input units
        dp_mask = self._dropout_mask
        # dropout matrices for recurrent units
        rec_dp_mask = self._recurrent_dropout_mask

        if self.implementation == 1:
            if 0. < self.dropout < 1.:
                inputs_z = inputs * dp_mask[0]
                inputs_r = inputs * dp_mask[1]
                inputs_h = inputs * dp_mask[2]
            else:
                inputs_z = inputs
                inputs_r = inputs
                inputs_h = inputs

            x_z = K.dot(inputs_z, self.kernel_z)
            x_r = K.dot(inputs_r, self.kernel_r)
            x_h = K.dot(inputs_h, self.kernel_h)
            if self.use_bias:
                x_z = K.bias_add(x_z, self.input_bias_z)
                x_r = K.bias_add(x_r, self.input_bias_r)
                x_h = K.bias_add(x_h, self.input_bias_h)

                
                
                
                
                
                
                
                
            if 0. < self.recurrent_dropout < 1.:
                h_tm1_z = h_tm1 * self.temporal_weight #rec_dp_mask[0]
                h_tm1_r = h_tm1 * self.temporal_weight #rec_dp_mask[1]
                h_tm1_h = h_tm1 * self.temporal_weight #rec_dp_mask[2]
            else:
                h_tm1_z = h_tm1 * self.temporal_weight
                h_tm1_r = h_tm1 * self.temporal_weight
                h_tm1_h = h_tm1 * self.temporal_weight

                
                
                
                
                
                
            recurrent_z = K.dot(h_tm1_z, self.recurrent_kernel_z)
            recurrent_r = K.dot(h_tm1_r, self.recurrent_kernel_r)
            if self.reset_after and self.use_bias:
                recurrent_z = K.bias_add(recurrent_z, self.recurrent_bias_z)
                recurrent_r = K.bias_add(recurrent_r, self.recurrent_bias_r)

            z = self.recurrent_activation(x_z + recurrent_z)
            r = self.recurrent_activation(x_r + recurrent_r)

            # reset gate applied after/before matrix multiplication
            if self.reset_after:
                recurrent_h = K.dot(h_tm1_h, self.recurrent_kernel_h)
                if self.use_bias:
                    recurrent_h = K.bias_add(recurrent_h, self.recurrent_bias_h)
                recurrent_h = r * recurrent_h
            else:
                recurrent_h = K.dot(r * h_tm1_h, self.recurrent_kernel_h)

            hh = self.activation(x_h + recurrent_h)
        else:
            if 0. < self.dropout < 1.:
                inputs *= dp_mask[0]

            # inputs projected by all gate matrices at once
            matrix_x = K.dot(inputs, self.kernel)
            if self.use_bias:
                # biases: bias_z_i, bias_r_i, bias_h_i
                matrix_x = K.bias_add(matrix_x, self.input_bias)
            x_z = matrix_x[:, :self.units]
            x_r = matrix_x[:, self.units: 2 * self.units]
            x_h = matrix_x[:, 2 * self.units:]

            if 0. < self.recurrent_dropout < 1.:
                h_tm1 *= rec_dp_mask[0]

            if self.reset_after:
                # hidden state projected by all gate matrices at once
                matrix_inner = K.dot(h_tm1, self.recurrent_kernel)
                if self.use_bias:
                    matrix_inner = K.bias_add(matrix_inner, self.recurrent_bias)
            else:
                # hidden state projected separately for update/reset and new
                matrix_inner = K.dot(h_tm1,
                                     self.recurrent_kernel[:, :2 * self.units])

            recurrent_z = matrix_inner[:, :self.units]
            recurrent_r = matrix_inner[:, self.units: 2 * self.units]

            z = self.recurrent_activation(x_z + recurrent_z)
            r = self.recurrent_activation(x_r + recurrent_r)

            if self.reset_after:
                recurrent_h = r * matrix_inner[:, 2 * self.units:]
            else:
                recurrent_h = K.dot(r * h_tm1,
                                    self.recurrent_kernel[:, 2 * self.units:])

            hh = self.activation(x_h + recurrent_h)

        # previous and candidate state mixed by update gate
        h = z * h_tm1 + (1 - z) * hh

        if 0 < self.dropout + self.recurrent_dropout:
            if training is None:
                h._uses_learning_phase = True

        return h, [h]


class CustomGRU(GRU):

    def __init__(self, units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True,
                 kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros',
                 kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None,
                 kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.,
                 recurrent_dropout=0., implementation=1, return_sequences=False, return_state=False, go_backwards=False,
                 stateful=False, unroll=False, reset_after=False, temporal_weight: float = 0.5, **kwargs):

        """
        super().__init__(units, activation=activation, recurrent_activation=recurrent_activation,
                         use_bias=use_bias, kernel_initializer=kernel_initializer, recurrent_initializer=recurrent_initializer,
                         bias_initializer=bias_initializer, kernel_regularizer=kernel_regularizer,
                         recurrent_regularizer=recurrent_regularizer, bias_regularizer=bias_regularizer,
                         activity_regularizer=activity_regularizer, kernel_constraint=kernel_constraint,
                         recurrent_constraint=recurrent_constraint, bias_constraint=bias_constraint,
                         dropout=dropout, recurrent_dropout=recurrent_dropout, implementation=implementation,
                         return_sequences=return_sequences, return_state=return_state, go_backwards=go_backwards,
                         stateful=stateful, unroll=unroll, reset_after=reset_after, **kwargs)
        """
        
        self.temporal_weight = temporal_weight

        cell = CustomGRUCell(units,
                       activation=activation,
                       recurrent_activation=recurrent_activation,
                       use_bias=use_bias,
                       kernel_initializer=kernel_initializer,
                       recurrent_initializer=recurrent_initializer,
                       bias_initializer=bias_initializer,
                       kernel_regularizer=kernel_regularizer,
                       recurrent_regularizer=recurrent_regularizer,
                       bias_regularizer=bias_regularizer,
                       kernel_constraint=kernel_constraint,
                       recurrent_constraint=recurrent_constraint,
                       bias_constraint=bias_constraint,
                       dropout=dropout,
                       recurrent_dropout=recurrent_dropout,
                       implementation=implementation,
                       reset_after=reset_after,
                       temporal_weight=temporal_weight)

        super(GRU, self).__init__(cell,
                                  return_sequences=return_sequences,
                                  return_state=return_state,
                                  go_backwards=go_backwards,
                                  stateful=stateful,
                                  unroll=unroll,
                                  **kwargs)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        
        
        
    def get_config(self):
        config = super().get_config()
        config["temporal_weight"] = self.temporal_weight
        return config
        
    def call(self, inputs, mask=None, training=None, initial_state=None):
        return super().call(inputs, mask, True, initial_state)
        
    

In [3]:
##### load the model and perform prediction
def useCustomGRU(dirPath, temporalWeight: float, timeLayer) -> Model:
    with open(dirPath + "_model.json", "r") as modelJsonFile:
        model = model_from_json(modelJsonFile.read())
    model.load_weights(dirPath + "_weight.h5py")
    #model.summary()
    
    #disasemble layers
    layers = [l for l in model.layers]

    # Get the trained forward layer from the bidirectional and change it's property
    b1 = model.get_layer("bidirectional_1")

    x = layers[0].output
    for i in range(1, len(layers)):
        if layers[i].name == "bidirectional_1":
            x = Bidirectional(
                CustomGRU(units=64, kernel_initializer='glorot_uniform', recurrent_dropout=0.8, dropout=0.0, return_sequences=True, temporal_weight=temporalWeight), name="custom_bi")(x)
        elif layers[i].name == "time_distributed_1":
            x = TimeDistributed( Dense(10, activation="sigmoid"), )(x)
        else:
            x = layers[i](x)

    newModel = Model(input=layers[0].input, output=x)
    #newModel.summary()
    model.load_weights(dirPath + "_weight.h5py")

    intermediate_model = Model(input=newModel.input, output=newModel.get_layer(timeLayer).output)
    
    return intermediate_model

# retreive inform,ation about the custom
#model = useCustomGRU(0.2)
#prediction = model.predict(featTest)




##### load the model and perform prediction
def useClassic(dirPath):
    with open(dirPath + "_model.json", "r") as modelJsonFile:
        model = model_from_json(modelJsonFile.read())
    model.load_weights(dirPath + "_weight.h5py")

    #model.summary()

    intermediate_model = Model(input=model.input, output=model.get_layer("time_distributed_1").output)
    #intermediate_model.summary()
    
    return intermediate_model

In [4]:
# Load evaluation set
# Load the test data create the test dataset
# load the file list
#featTestPath = "/baie/corpus/DCASE2018/task4/FEATURES_16K/test/mel"
featTestPath = "/baie/corpus/DCASE2018/task4/FEATURES/test/mel"
featTestList = os.listdir(featTestPath)

# load the meta data ----
metaPath = "/baie/corpus/DCASE2018/task4/metadata/test.csv"
with open(metaPath, "r") as metaFile:
    metadata = metaFile.read().splitlines()[1:]
    
metadata = [i.split("\t") for i in metadata]

# load the features
featTest = []
for file in tqdm.tqdm(featTestList):
    path = os.path.join(featTestPath, file)
    feature = np.load(path)
       
    # preprocessing
    feature = np.expand_dims(feature, axis=-1)
    featTest.append(feature)
    
featTest = np.array(featTest)

100%|██████████| 279/279 [00:00<00:00, 1464.81it/s]


In [9]:
K.clear_session()
# remove GAP layer on last network. duplicate and change GRU by WGRU for one
#tModel = useClassic("results/complete_2")
#t20Model = useCustomGRU("results/complete_2", 0.25, "time_distributed_1")
#t10Model = useCustomGRU("results/complete_2", 0.10, "time_distributed_2")
#results/testing/mel_old_noBlank_timeReduction2/oldMel_noBlank_reduce2
tModel = useClassic("results/testing/mel_old_noBlank_timeReduction2/oldMel_noBlank_reduce2")
t20Model = useCustomGRU("results/testing/mel_old_noBlank_timeReduction2/oldMel_noBlank_reduce2", 0.25, "time_distributed_1")
t10Model = useCustomGRU("results/testing/mel_old_noBlank_timeReduction2/oldMel_noBlank_reduce2", 0.10, "time_distributed_2")

/homeLocal/eriador/.miniconda3/envs/dcase/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ti...)`


Temporal weight :  0.25
Temporal weight :  0.25


/homeLocal/eriador/.miniconda3/envs/dcase/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`
/homeLocal/eriador/.miniconda3/envs/dcase/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ti...)`


Temporal weight :  0.1
Temporal weight :  0.1


In [10]:
print("gru temporal prediction ...")
tPrediction = tModel.predict(featTest)
print("wgru temporal prediction")
t20Prediction = t20Model.predict(featTest)
t10Prediction = t10Model.predict(featTest)
nbFrame = tPrediction.shape[1]
print(tPrediction.shape)

# mix the prediction giving the globals prediction
w20gru_cls = [0, 2, 1]               # "impulse" event well detected by the WGRU
w10gru_cls = []
gru_cls   = [8, 3, 5, 7, 6, 9, 4]   # "stationary" event well detected by the GRU

finalTPrediction = []
#for i, gp in enumerate(gbPrediction):
#    cls = gp.nonzero()[0]
for i in range(len(tPrediction)):

    curves = np.array([[0]*10 for _ in range(nbFrame)], dtype=np.float32)

    # use the WGRU temporal
    for c in w20gru_cls:
        curves[:,c] = t20Prediction[i][:,c] # use the wgru temporal prediction
        
    for c in w10gru_cls:
        curves[:,c] = t10Prediction[i][:,c] # use the wgru temporal prediction

    for c in gru_cls:
        curves[:,c] = tPrediction[i][:,c]  # use the classic gru temporal prediction

    finalTPrediction.append(curves)
finalTPrediction = np.array(finalTPrediction)
print(finalTPrediction.shape)

encoder = Encoder()
segments = encoder.encode(finalTPrediction, method="threshold", smooth="smoothMovingAvg")
toEvaluate = encoder.parse(segments, featTestList)

print("perform evaluation ...")
with open("toEvaluate.csv", "w") as f:
    f.write("filename\tonset\toffset\tevent_label\n")
    f.write(toEvaluate)

gru temporal prediction ...
wgru temporal prediction
(279, 215, 10)
(279, 215, 10)
Smooting using the smooth moving average algorithm
perform evaluation ...


In [11]:
perso_event_list = MetaDataContainer()
perso_event_list.load(filename="toEvaluate.csv")

ref_event_list = MetaDataContainer()
ref_event_list.load(filename="meta/test.csv")

event_based_metric = event_based_evaluation(ref_event_list, perso_event_list)
print(event_based_metric)

Event based metrics (onset-offset)
  Evaluated length                  : 2616.88 sec
  Evaluated files                   : 288 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 200.00 ms
  Offset (length)                   : 20.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 3.58 %
    Precision                       : 2.45 %
    Recall                          : 6.62 %
  Error rate
    Error rate (ER)                 : 3.48 
    Substitution rate               : 0.09 
    Deletion rate                   : 0.84 
    Insertion rate                  : 2.55 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 9.28 %
    Precision                       : 8.31 %
    Recall                          : 12.64 %
  Error rate
    Error rate (ER)                 : 3.06 
    Deletion rate                   : 0.87 
    Insertion rate      

In [8]:
encoder = Encoder()
segments = encoder.encode(t10Prediction, method="threshold", smooth="smoothMovingAvg", window_len=19)
toEvaluate = encoder.parse(segments, featTestList)

print("perform evaluation ...")
with open("toEvaluate.csv", "w") as f:
    f.write("filename\tonset\toffset\tevent_label\n")
    f.write(toEvaluate)
    
perso_event_list = MetaDataContainer()
perso_event_list.load(filename="toEvaluate.csv")

ref_event_list = MetaDataContainer()
ref_event_list.load(filename="meta/test.csv")

event_based_metric = event_based_evaluation(ref_event_list, perso_event_list)
print(event_based_metric)

Smooting using the smooth moving average algorithm
perform evaluation ...
Event based metrics (onset-offset)
  Evaluated length                  : 2616.88 sec
  Evaluated files                   : 288 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 200.00 ms
  Offset (length)                   : 20.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 2.37 %
    Precision                       : 1.36 %
    Recall                          : 9.05 %
  Error rate
    Error rate (ER)                 : 7.03 
    Substitution rate               : 0.42 
    Deletion rate                   : 0.49 
    Insertion rate                  : 6.12 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 1.76 %
    Precision                       : 1.05 %
    Recall                          : 10.91 %
  Error rate
    Error rate (ER)                 : 